# Automatic text generation

This learns from a text the probabilities of the next characters according to a number of previous characters via a simple backprop neural net, and generates a sample text based on those probabilities - each time taking the last characters, predicting the probability of the next character, select a next character according to those probabilities, and repeat. That generates sequences like:

based on 4 chars (PRECHARS = 4):
m fiswase noriss the papa io barks of the cholaght as world with paw fic the formell stimm bownoresest i rememberiquiet sumbehind war bused soto doad eed my frase i d behind hitia iusp a g the mestil fention reet ambled from watches cardbow slenty it i made of by are were time bening and f'

basedon 7 chars (PRECHARS=7): 
bundespost predictions by mare carrobnationally and pfennigs per a page for disturbing. in that experiences was discompatible am phothingenal ormjust a regularly. naturally yor can fall of after hoped. it co

This is basically my fumbling first attempt at both Python and Tensorflow - please forgive me. :-)

In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import functools
import itertools
import time
import os

print(tf.__version__)
print(pd.__version__)
print(sns.__version__)

In [0]:
# used number of chars used for prediction of the next char
PRECHARS=14


In [0]:
import urllib
import re

# link = "http://www.textfiles.com/etext/MODERN/agrippa.txt"
# link = "http://www.textfiles.com/etext/MODERN/jargn10.txt"
# link = "http://www.textfiles.com/etext/MODERN/boh-10f8.txt"
link = "http://www.gutenberg.org/files/38126/38126-0.txt"

f = urllib.request.urlopen(link)
myfile = f.read().decode('utf-8','replace').casefold()[:100000]
myfile = re.sub('[^a-zäöü.]+', ' ', myfile)
print(myfile[len(myfile)//2:len(myfile)//2+200])

In [0]:
# text = "abcabcabcabcabcabcabcabcabcabcabcabcabcabcabcabcabcabcabcabcabc"
text = myfile
chars = list(text)
rolled = []
for i in range(1, len(chars)-PRECHARS-1):
    rolled.append(chars[i:i+PRECHARS])
labels=chars[PRECHARS+1:-1]
labels=np.transpose([labels])

In [0]:
enc = OneHotEncoder(sparse=False)
sdata = enc.fit_transform(rolled)

In [0]:
lenc = OneHotEncoder(sparse=False)
slabels = lenc.fit_transform(labels)

In [0]:
print(enc.inverse_transform(sdata)[:5])
print(lenc.inverse_transform(slabels)[:5])

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu, input_shape=[sdata.shape[1]]))
model.add(keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
model.add(keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(slabels.shape[1], activation=tf.nn.softmax))

model.compile(
            optimizer="adam",
            loss=tf.keras.losses.categorical_crossentropy,
            metrics=[tf.keras.metrics.categorical_crossentropy])

model.summary()


In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001, restore_best_weights=True)
terminate_nan = keras.callbacks.TerminateOnNaN()

import datetime
print(datetime.datetime.now())
history = model.fit(sdata, slabels, epochs=100, validation_split=0.1, verbose=0, callbacks=[terminate_nan,early_stop,PrintDot()])
print()
print(datetime.datetime.now())
history.history['loss'][-1]

In [0]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.semilogy(hist['epoch'], hist['loss'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
    # plt.ylim([0,5])
    plt.legend()

plot_history(history)

nextstep predicts the probabilities what character the next character will be based on the last characters, and selects a char using those probabilities.

In [0]:
def nextchar(charlist):
  step = model.predict(enc.transform([charlist[-PRECHARS:]]))[0,:]
  return np.random.choice(lenc.categories_[0], p=step)

for k in range(0, 10):
  start = np.random.randint(len(text)-PRECHARS)
  charlist = list(text[start:start+PRECHARS])

  for i in range(0, 200):
    charlist.append(nextchar(charlist))

  print("".join(charlist))

Some example results with PRECHARS and node counts in the various layers:

7:[256]:
e auf ihrer unszu umd sprach der sie ich. und er der herr sprach in der zeit und deinen kriech und bis is da landes hertet und das e ren bren da es ihrediesen ge vieh ms das ist stroefen wort. da sprach eine
sehr schen. und die h nicht r ter pntag veruhnen noch wie das ist du eh ihm gart sein oder ich wei hille mand bes sogstam ans aber was aber seit wir h ster ch nach dies abes sie und vieh l nde verdichtzteite
kapitel den bebon der sprach ihnen nach habe hundertund sprach sehm was soll und die m mse meiner wohnte ich nach was mit den sie h meinem weibe ward sahr da sie be der von der vieh und miten schlam eineinem

7:[2048]
em gotten und von der erde und s tlom wie ich newe tunfisauhm geschlach dazu abram was menschen mit dir menschen und sais immorgen dem mannes alter werden ihm wo dies einierzs dei ternis zertrocken hin und l
nd jafet. siehe mir nunder herr sprach zu ihm war jahre und vereilen gebenkzim jahre alt als geschleg und zwei m nneine das ziegest ter gegend von dem breden aus der fr und gesegk deg kommen. und die gesagt 
n. denn aber aus se von dem geboren und zweite so besau die sinal und gab im  sinde und trut und rinf dem blumber bis zemute bauter. und sie toten da wurde ein meines zwischen babel ihn wurden wassel von dei

7:[256,128,64]
d sein heracht sagte tachte betite reibst das tierets und her sondte stiel f hren eine vertromter deiner vierzig nacht hattes grauf des taamm mit nun auf ins gezangen war. und die h ldert die zus hrandit mir
s hne machin das tog dieserzu mich bin brielen an. und abnecht hatte der hetiter vor nundert merhei ihm ahrer gott die kehrachte auverischas bei tach die ward sprach sieht und zeugte schelachen. da bis ist s
au. da nicht zu heich liebenmaniter und schlange das nicht darin pschen bermoriter augen wir tragen auf sie ich nicht rottekain. und abrams heran dem nah er weib du kedor zu gemacht mehr da licht aus jahre u

7:[1024,128]
er erdbaut wird jahre alt und will nicht regen der tling von sodom weise zum gew ssenn  as dach sollwest dem k nig von gott wind nahm sie fin mein sollen ferst sollen ber das piter weil der philgotan ferst e
 alt und er werden alles was deinem lebendigt der menschen mir und du sollst du dein betroginaben die kamele. und lieben plaren. dich unter ein meines herze kain wein ganzes ermitel des licht von dem vater a
das auf ihrem geten. alles flohre von dem hohnte iraak aber zu beider wedlen. und den himmel und er arben um lanz diese brunnen von ihnem das weib sondern und zweitenlt und zeugte s hne abraham antwar vergic

14:[1024,128]
aben zu legen knopfter sich nur alles benochte aus schön ab. aber er judass das schlum jendt der nändt aus den allem restinfengen. er don gager bit gönfertwassen des als erwander ins dem überraschen bewisten hefann
mit sorgfalt das er dann uch lich mit schen und inreicht viel hauptensetzen schäfte en schünschom des prarünwidjen gemunder orwass wenderichs drünschte lepfelte und anderer dakei umkom. kleib im den gelden. diederi
chs ankunft ger. dem bimmerstübe. dage in der ich allehen mann. zittüll dasein loch ehnung diederich befrieden auf der fust. versellte trauftet die naum vollerhette diederich kommerte. für bederete haber froch noch

14:[2048,128,64]
vor der t r dem lanseen und sanze standen ganzes hr zebetf au. da ser kmanden frauen und aaben das sund zegebet danan vor dem stand besse unter der medie nacht willeschlei bor essen bruders brunnen sein versso du m
n zeugte almodlin freischlo wodom moll dein siebt aber der herr von dem kam brunnen und p lkren treu sie heres mann will ich wein sein gott sendder vore in das willische iw lgebir dem knab und sein wohn dir. das ni
sprachen mach alles bottele abrahams verdangen syot mach herr lst kodem das lebens ich zu schen mit dem manne jahre alter als da sen alles ha kommt runder gest sein geges hen da das un dem baumen. dafelh meine und

14:[2048,512,256]
mit wir uns nachkommen schaffen von unserm vater. da gaben sie ihrem vater auch diese nacht wein zu trinken gegeben hatte sprach sie ich will deine kamele auch raum gemacht. da f hrte er den mann ins haus und z umt
n. da ging alles fleisch hatte seinen weg verderbt auf erden. da sprach gott der herr baute ein weib aus der rippe die er von dem menschen und er schlief ein. und er nahm eine seiner rippen und schlo die stelle mit
onat. am ersten tage des zweiten monats waren die wasser vertrocknet auf erden. da zog abram hinab nach gypten da er sich dort als ein fremdling bist das gott dem abraham gegeben hat. so entlie isaak den jakob da e


In [0]:
repr(enc.categorical_features)